In [7]:
from data_sampler import WeightedSampler
from dataset import CARLADataset, CARLADatasetMultiProcessing
from data_preprocessing import preprocessing
from torch.utils.data import DataLoader
from torchvision import transforms
import numpy as np
import torch
import os
import pandas as pd
import matplotlib.pyplot as plt

from tqdm import tqdm

In [4]:
# path_data = os.path.join("..", "data", "Dataset Ege")
path_data = os.path.join("..", "data", "data")

config = {"used_inputs": ["rgb", "measurements"], 
        "used_measurements": ["speed", "steer", "throttle"],
        "seq_len": 1
        }

dataset = CARLADataset(root_dir=path_data, config=config)
# dataset_mp = CARLADatasetMultiProcessing(root_dir=path_data, config=config)
weighted_sampler = WeightedSampler(dataset=dataset)
print(dataset.__len__())

258866


In [5]:
dl = DataLoader(dataset=dataset, batch_size=16, num_workers=0, sampler=weighted_sampler)
# dl_mp = DataLoader(dataset=dataset_mp, batch_size=16, num_workers=3, sampler=weighted_sampler)

In [9]:
b = next(iter(dl))

In [8]:
count = 0
for batch in tqdm(dl):
    count += 1
print(count)

100%|██████████| 16180/16180 [48:35<00:00,  5.55it/s]

16180


In [4]:
count = 0
for batch in dl:
    # preprocessing
    for key in preprocessing:
        batch[key] = preprocessing[key](batch[key])
    count += 1
print(count)

14
